In [ ]:
# WARNING: advised to install a specific version, e.g. tensorwaves==0.1.2
%pip install -q tensorwaves[doc,jax,pwa,viz] IPython

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

from IPython.display import display  # noqa: F401

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Binned fit

Imagine we have the following data distribution over $x$:

In [ ]:
import numpy as np

sample_size = 1_000
rng = np.random.default_rng(seed=0)
x_distribution = rng.power(3, size=sample_size)

:::{margin}

Alternatively, we could have sampled the distribution with {func}`numpy.histogram`.

:::

We sample the distribution into a histogram with {func}`matplotlib.pyplot.hist`. The bin edges will serve as $x$-values and the bin heights will be the observed $y$-values.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 5))
ax.set_xlabel("$x$")
n_bins = 50
bin_values, bin_edges, _ = ax.hist(x_distribution, bins=n_bins)
plt.show()

In [ ]:
x_values = (bin_edges[1:] + bin_edges[:-1]) / 2
y_values = bin_values

The data distribution has been generated by {func}`numpy.random.power` and can therefore be described by the following expression:

In [ ]:
import sympy as sp

n, a, x = sp.symbols("n a x")
expression = n * x ** (a - 1)
expression

We would like to find which values for $a, n$ describe this distribution best. For fast computations, we first formulate this expression as a {class}`.ParametrizedFunction` with {mod}`numpy` as computational backend ({mod}`numpy` is fast enough for the small data samples with which we are working here). We also provide a first guess for the values of $a$ and $n$.

In [ ]:
from tensorwaves.function.sympy import create_parametrized_function

function = create_parametrized_function(
    expression,
    parameters={a: 2, n: 15},
    backend="numpy",
)
initial_parameters = function.parameters

In [ ]:
function.update_parameters(initial_parameters)
fig, ax = plt.subplots(figsize=(8, 5))
ax.set_xlabel("$x$")
ax.hist(x_distribution, bins=n_bins, label="Data distribution")
ax.plot(
    x_values, function({"x": x_values}), label="Initial fit model", c="red"
)
ax.legend()
plt.show()

This **binned distribution** lends itself well to be optimized with a {class}`.ChiSquared` estimator.

In [ ]:
from tensorwaves.estimator import ChiSquared
from tensorwaves.optimizer import Minuit2

estimator = ChiSquared(
    function,
    domain={"x": x_values},
    observed_values=y_values,
    backend="numpy",
)
optimizer = Minuit2()
fit_result = optimizer.optimize(estimator, initial_parameters)
fit_result

In [ ]:
assert fit_result.minimum_valid

The optimized value of $a$ indeed lies close to the value of $a=3$ with which we generated the distribution.

In [ ]:
function.update_parameters(fit_result.parameter_values)
fig, ax = plt.subplots(figsize=(8, 5))
ax.set_xlabel("$x$")
ax.hist(x_distribution, bins=n_bins, label="Data distribution")
ax.plot(x_values, function({"x": x_values}), label="Optimized model", c="red")
ax.legend()
plt.show()

Also note that the integral over the expression with these optimized parameter values is close to the surface of the bins:

In [ ]:
bin_width = bin_edges[1] - bin_edges[0]
surface = bin_values.sum() * bin_width
surface

In [ ]:
from IPython.display import Math

substitutions = {
    a: fit_result.parameter_values["a"],
    n: fit_result.parameter_values["n"],
}
integral = sp.Integral(expression, (x, 0, 1))
evaluated_integral = integral.subs(substitutions).doit()
latex = sp.multiline_latex(
    integral, evaluated_integral, environment="eqnarray"
)
Math(latex)